# Example 10-2. 한글의 음소 단위 문장 생성 모형 

- 한글 텍스트를 음소 단위로 토큰화 하여 문장을 생성하는 모형을 만들어 보는 예제 
- 특정 형식과 어투를 가지고 있는 텍스트(e.g. 구약성경)를 사용하여 모형 학습 

## 데이터 불러오기 

- 한글 데이터를 한 줄씩 읽어와서 음소 단위로 나눔

In [ ]:
!pip install jamotools

In [ ]:
import jamotools

In [ ]:
INPUT_FILE = "./data/oldTestament.txt"
# extract the input as a stream of characters
print("Extracting text from input...")
fin = open(INPUT_FILE, 'rb')

lines = []
i=0
for line in fin:
    line = line.strip()
    line = line.decode("utf-8")
    line = jamotools.split_syllables(line) 
    line = line + '\n' 
    if len(line) == 0:
        continue
    lines.append(line)
    i += 1
    if i == 2000: # 2000줄만 불러오기 
        break
fin.close()

text = " ".join(lines)

In [ ]:
len(lines)

<font color=blue>

TO DO: 

- 유일한 문자의 집합인  vocabulary를 생성하여 `chars` 이름으로 저장하시오. 
- vocabulary의 크기를 `nb_chars`에 저장하시오. 


In [ ]:
chars = 
nb_chars = 


<font color=blue>
    
TO DO:

- char와 index를 연결하는 lookup table을 만드시오. (`char2index`, `index2char`)

In [ ]:
# Your answer comes here





## 데이터 준비하기

<font color=blue>
    
TO DO:

- 20개의 음소를 입력값으로, 다음에 오는 음소를 출력값으로 하는 텍스트 list를 만드시오. 
- 참고: Lecture10의 "Preparing data" 부분을 활용 
 
         input_chars[:5]
         ['ㅊㅏㅇ1:1 <ㅊㅓㄴㅈㅣ ㅊㅏㅇㅈㅗ>',
         '1:1 <ㅊㅓㄴㅈㅣ ㅊㅏㅇㅈㅗ> ㅌㅐ',
         ' <ㅊㅓㄴㅈㅣ ㅊㅏㅇㅈㅗ> ㅌㅐㅊㅗㅇ',
         'ㅓㄴㅈㅣ ㅊㅏㅇㅈㅗ> ㅌㅐㅊㅗㅇㅔ ㅎ',
         'ㅣ ㅊㅏㅇㅈㅗ> ㅌㅐㅊㅗㅇㅔ ㅎㅏㄴㅏ']
         
         label_chars[:5]
         [' ', 'ㅊ', 'ㅔ', 'ㅏ', 'ㄴ']


In [ ]:
SEQLEN = 20
STEP = 3

input_chars = []
label_chars = []
 
for i in range(0, len(text) - SEQLEN, STEP):
    # Your answer comes here





In [ ]:
input_chars[:5]

In [ ]:
label_chars[:5]

<font color=blue>
    
TO DO:

- 위에서 만든 input/output 문자 셋을 one-hot encoding으로 변환하여 모형에 입력 가능한 형태로 변환

        X.shape, y.shape
        ((83870, 20, 67), (83870, 67))

In [ ]:
import numpy as np 

X = np.zeros((len(input_chars), SEQLEN, nb_chars), dtype=np.bool)
y = np.zeros((len(input_chars), nb_chars), dtype=np.bool)

for i, input_char in enumerate(input_chars):
    # Your answer comes here





In [ ]:
X.shape, y.shape

## 모델 학습

<font color=blue>
    
TO DO: 

- 20개의 음소를 입력하여 그 다음에 나타날 음소를 예측하는 LSTM 모형을 생성하시오. 
- HIDDEN_SIZE, layer의 수 등 hyperparameter를 바꾸어 가며 최적의 모형을 찾으시오. 

In [ ]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras import optimizers
from keras import backend as K
HIDDEN_SIZE = 30
BATCH_SIZE = 128

K.clear_session()
# Your answer comes here





## 문장 생성

In [ ]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

<font color='blue'>

TO DO: 
             
- 첫 입력데이터 `창1:1 <천지 창조>`를 seed로 하여 이후의 400개 음소를 생성하고 이 음소를 다시 글자로 조합하여 문장 생성 테스트를 진행하시오. 
- `sample`함수의 `temeprature`를 조정하여 동일 seed에서 생성되는 문장의 차이를 비교하시오. 

In [ ]:
## random seed의 사용 

#test_idx = np.random.randint(len(input_chars))
#test_chars = input_chars[test_idx]

## 첫 번째 seed를 사용 
test_chars = input_chars[0]
print("Generating from seed: %s" % (jamotools.join_jamos(test_chars)))
print(test_chars, end="")
ypred_join=test_chars
for i in range(400):
    Xtest = np.zeros((1, SEQLEN, nb_chars))
    for i, ch in enumerate(test_chars):
        Xtest[0, i, char2index[ch]] = 1
    pred = model.predict(Xtest, verbose=0)[0]
    ypred = index2char[sample(pred,0.3)]
    ypred_join = ypred_join + ypred 
    # move forward with test_chars + ypred
    test_chars = test_chars[1:] + ypred

jamotools.join_jamos(ypred_join)